In [11]:
import glob
import rasterio
import rasterio.features
import rasterio.warp
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from tqdm import tqdm

In [9]:
SWISSIMAGE_PATH = './dataset/base'
LULC_PATH = './dataset/lulc'
SWISSALTI3D_PATH = './dataset/alti'

In [7]:
def get_lulc_cmap(n_classes=17):
    cmap = plt.cm.jet
    cmaplist = [cmap(i) for i in range(cmap.N)]

    bounds = np.linspace(0, n_classes, n_classes+1)
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

    cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)

    return cmap, norm

In [21]:
def plot_map(base_map, lulc_tile, alti_tile):
    plt.figure(figsize=(10,10))
    
    plt.subplot(1,3,1)
    plt.title('Aerial Image')
    ax = plt.gca()
    ax.axis('off')
    im = ax.imshow(base_map)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    cb = plt.colorbar(im, cax=cax)
    cb.remove() 


    cmap, norm = get_lulc_cmap(n_classes=72)

    plt.subplot(1,3,2)
    plt.title('LU / LC Map')
    ax = plt.gca()
    ax.axis('off')
    im = ax.imshow(lulc_tile, alpha=1, cmap=cmap, norm=norm)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax)

    plt.subplot(1,3,3)
    plt.title('Altitude Map')
    ax = plt.gca()
    ax.axis('off')
    im = ax.imshow(alti_tile, alpha=1)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax)

    plt.tight_layout()
    plt.show()

In [14]:
def read_base_map_tile(path):
    with rasterio.open(path) as dataset:
        tile_bounds = dataset.bounds
        r = dataset.read(1)
        g = dataset.read(2)
        b = dataset.read(3)
        rgb_base_image = np.stack((r, g, b), axis=-1)

    return tile_bounds, rgb_base_image

def read_alti3d_tile_2(east, north):
    alit3d_files = glob.glob(f'{SWISSALTI3D_PATH}/swissalti3d_*_{east}-{north}_*.tif')
    if len(alit3d_files) != 1:
        print('ERROR: ', alit3d_files, east, north)
        return np.full((500, 500), -9999)

    with rasterio.open(alit3d_files[0]) as dataset:
        tile = dataset.read(1)

    return tile

def read_lulc_tile(east, north):
    lulc_tiles = glob.glob(f'{LULC_PATH}/{east}_{north}.npy')
    if len(lulc_tiles) != 1:
        print('ERROR: ', lulc_tiles, east, north)
        return np.full((40, 40), -9999)

    return np.load(lulc_tiles[0])

In [ ]:
base_maps = glob.glob(f"{SWISSIMAGE_PATH}/**.tif")
for base_map_tile_path in tqdm(base_maps[0:10]):
    base_map_info = base_map_tile_path.split('_')
    coords = base_map_info[2].split('-')
    east, north = coords

    bounds, base_image = read_base_map_tile(base_map_tile_path)
    lulc_tile = read_lulc_tile(east, north)
    alti_image = read_alti3d_tile_2(east, north)

    plot_map(base_image, lulc_tile, alti_image)